In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import normalize
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
pip install tensorflow

In [ ]:
original_data = pd.read_csv("C:\\Users\\Admin\\Desktop\\upwork_jobs\\lol_project\\data\\high_diamond_ranked_10min.csv")

In [ ]:
required_columns = [
                    ## ID and Predictor Variable
                    'blueWins',
                    
                    ## Statistics for Blue Teams
                    'blueWardsPlaced', 'blueWardsDestroyed','blueKills', 'blueDeaths',
                    'blueAssists','blueEliteMonsters', 'blueDragons', 'blueHeralds',
                    'blueTowersDestroyed', 'blueAvgLevel','blueCSPerMin', 'blueGoldPerMin', 
                    
                    ## Shared Statisitics
                    'blueGoldDiff', 'blueExperienceDiff','blueFirstBlood',
                    
                    ## Statistics for Red Teams
                    'redWardsPlaced', 'redWardsDestroyed','redKills', 'redDeaths',
                    'redAssists','redEliteMonsters', 'redDragons','redHeralds',
                    'redTowersDestroyed', 'redAvgLevel','redCSPerMin','redGoldPerMin'
                    ]

In [ ]:
modelling_df = original_data[required_columns]

In [ ]:
modelling_df.head()

In [ ]:
## Logistic Modelling
X = modelling_df[required_columns[1:]].values
Y = modelling_df['blueWins'].values

In [ ]:
# Split dataset to train and test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.2, random_state=42)

In [ ]:
# Normalize and change data type to floate32
x_train = normalize(x_train, axis=1).astype('float32')
x_test = normalize(x_test, axis=1).astype('float32')

In [ ]:
# Baseline model for the neural network. We choose a hidden layer of 10 neurons. The lesser number of neurons helps to eliminate the redundancies in the data and select the more important features.
def create_baseline():
    # create model
    model = Sequential([
        Dense(64, input_shape=(x_train.shape[1],), activation='relu'),
        Dense(32, activation='relu'),
#         Dropout(0.25),
        Dense(1)      
    ])
    
    return model

In [ ]:
# Compile model
base_model = create_baseline()

base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
epochs = 20
# Fit model
history = base_model.fit(x_train, y_train,
               validation_data=(x_test, y_test), batch_size=8, epochs = epochs )

In [ ]:
pred = base_model.predict(x_test)
score = tf.nn.sigmoid(pred)

# convert predictions from probabilty to integers
predictions = np.where(pred < 0.5, 0, 1) # use 0.5 as threshold, could change

In [ ]:
#test
predictions[0], y_test[0]

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(loss, label='Training loss')
plt.plot(val_loss, label='Validation loss')
plt.xlabel('Epochs')
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f"Model Accuracy: {round(accuracy, 2)}\nModel F1 Score: {round(f1,2)}")